# Q Learning

In [2]:
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib import animation, rc

Q function represents the discounted future reward from a starting state.
By adopting an optimal action policy, actions can be taken at each state to achieve the highest Q value at the end.

# Deterministic MDPs

Intuitively, for RL, for a given state, we would like to choose a action to benefit all future states to eventually reach the winning point. Such benefit can be formulated as a cumulative decremental reward for a starting state at the instant $t$:

$$ V^{\pi}(s,t) =  \sum_{i=0}^N \gamma^i r_{t+i}$$

The goal is to find an optimal policy $\pi^*$ that maximizes the cumulative reward:

$$ \pi^*(s) = argmax_a V^{\pi}(s)$$


However, in reality, it is hard to have all reward at each state in advance to compute the value $V$.
So we can consider the optimal policy will be the one that maximize the sum of the immadiate reward resulting an action and the value of all subsequent states after the action, which results in a iterative form of Q learning:

$$Q(s,a) = r + \gamma Max(Q(s',a'))$$

where
 * $Q(s,a)$ is the Q value of the current state
 * $r$ is the reward by taking action $a$ from the current state
 * $Q(s',a')$ is the Q value of the next state

### Pseudo-code

```
initialize Q[numstates,numactions] arbitrarily
observe initial state s
repeat
    select and carry out an action a
    observe reward r and new state s'
    Update Q table by
        Q[s,a] = r + γmax Q[s',a']
    s = s'
until terminated
```

In [63]:
# S F F F       (S: starting point, safe)
# F H F H       (F: frozen surface, safe)
# F F F H       (H: hole, stuck forever)
# H F F G       (G: goal, safe)

env = gym.make("FrozenLake-v1", is_slippery=False, render_mode="ansi")

In [64]:
out = env.reset()
print(out)
out= env.step(2)
print(out)
print(env.render())

(0, {'prob': 1})
(1, 0.0, False, False, {'prob': 1.0})
  (Right)
SFFF
FHFH
FFFH
HFFG



In [75]:
def test(env, num, Q=None):
    episodes = 100
    nb_success = 0

    # Evaluation
    for _ in range(num):
        state, _ = env.reset()
        done = False

        # Until the agent gets stuck or reaches the goal, keep training it
        while not done:
            # Choose the action with the highest value in the current state
            if Q is not None:
                action = np.argmax(Q[state])
            else:
                action = env.action_space.sample()

            # Implement this action and move the agent in the desired direction
            new_state, reward, done, _, _ = env.step(action)

            # Update our current state
            state = new_state

            # When we get a reward, it means we solved the game
            nb_success += reward

    # Let's check our success rate!
    print (f"Success rate = {nb_success/num*100}%")

test(env, 100)

Success rate = 1.0%


In [52]:
#Initialize Q table of dimension (16 states X 4 actions)
Q = np.zeros([env.observation_space.n,env.action_space.n])
# Set learning parameters
gamma = 0.95
alpha = 0.5 
num_episodes = 1000
done = False
#create lists to contain total rewards and steps per episode
#jList = []
rList = []
for i in range(num_episodes):
    #Reset environment and get first new observation
    s, _ = env.reset()
    rAll = 0
    done = False # fall in H
    o = False # invalid action

    #The Q-Table learning algorithm
    while not done:
        if np.max(Q[s]) > 0:
            #Choose an action by greedily (with noise) picking from Q table
            a = np.argmax(Q[s])
        else:
            # else select ramdomly
            a = env.action_space.sample()

        # apply action and get new state and reward from environment
        s1, r, done ,_, _ = env.step(a)

        #Update Q-Table with new knowledge
        Q[s,a] = (r + gamma*np.max(Q[s1,:]))

        rAll += r
        s = s1

    #jList.append(j)
    rList.append(rAll)

In [53]:
print("Final Q-Table Values")
print(Q)

Final Q-Table Values
[[0.         0.77378094 0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.81450625 0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.857375   0.        ]
 [0.         0.         0.9025     0.        ]
 [0.         0.95       0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         1.         0.        ]
 [0.         0.         0.         0.        ]]


The Q values change at each run, which due to the randomness of the game (a wind blow may change randomly the state).

In [54]:
test(env, 100, Q)

Success rate = 100.0%


# Nondeterministic MDP

This modified equation is adapted to the randomness of the problem with which the Q value is approximated by the Q value of previous iteration.

$$Q(s,a) = E[r] + \gamma \sum p(s'/ s,a ) Max(Q(s',a'))$$
$$Q_n(s,a) = (1-\alpha_n)Q_{n-1}(s,a) + \alpha_n [r + \gamma Max(Q_{n-1}(s',a'))]$$

with $$\alpha_n = \frac{1}{1+Visits_n(s,a)}$$

where
 * $Q(s,a)$ is the Q value of the current state
 * $r$ is the reward by taking action $a$ from the current state
 * $Q(s',a')$ is the Q value of the next state
 * $\alpha$ is the learning rate
 * $Visit$ is the iteration number

### Pseudo-code
```
initialize Q[numstates,numactions] arbitrarily
observe initial state s
repeat
    select and carry out an action a
    observe reward r and new state s'
    Update Q table by
        Q[s,a] = Q[s,a] + α(r + γmaxa' Q[s',a'] - Q[s,a])
    s = s'
until terminated
```

In [58]:
# Even this example doesn't make sense here, it is served only as a benchmark to test our method.

#Initialize table with all zeros
Q = np.zeros([env.observation_space.n,env.action_space.n])
visits = np.ones([env.observation_space.n,env.action_space.n])
# Set learning parameters
gamma = 0.95
num_episodes = 1000
#create lists to contain total rewards and steps per episode
#jList = []
rList = []
for i in range(num_episodes):
    #Reset environment and get first new observation
    s, _ = env.reset()
    rAll = 0
    done = False # fall in H
    #The Q-Table learning algorithm
    while not done:
        if np.max(Q[s]) > 0:
            #Choose an action by greedily (with noise) picking from Q table
            a = np.argmax(Q[s])
        else:
            # else select ramdomly
            a = env.action_space.sample()

        #Get new state and reward from environment
        s1, r, done, _, _ = env.step(a)

        #Update Q-Table with new knowledge
        alpha = 1.0 / visits[s, a]
        Q[s,a] = (1-alpha)*Q[s,a] + alpha*(r+gamma*np.max(Q[s1,:]))

        rAll += r
        s = s1
        visits[s, a] += 1

    #jList.append(j)
    rList.append(rAll)

In [59]:
print("Final Q-Table Values")
print(Q)

Final Q-Table Values
[[0.08163602 0.2304565  0.13905771 0.08301543]
 [0.14684164 0.         0.09963049 0.06880982]
 [0.08723636 0.2974211  0.0874676  0.13386757]
 [0.28255005 0.         0.06508916 0.13674933]
 [0.09162589 0.24258579 0.         0.15509241]
 [0.         0.         0.         0.        ]
 [0.         0.31307485 0.         0.15611931]
 [0.         0.         0.         0.        ]
 [0.38566212 0.         0.81983764 0.2292585 ]
 [0.59323588 0.86298699 0.22910044 0.        ]
 [0.81983764 0.47251511 0.         0.17276561]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.18021446 0.90840735 0.81983764]
 [0.86298699 0.34540082 0.95913705 0.77884576]
 [0.         0.         0.         0.        ]]


In [60]:
test(env, 100, Q)

Success rate = 100.0%


## Epsilon-greedy

In [79]:
#Initialize table with all zeros
Q = np.zeros([env.observation_space.n,env.action_space.n])
visits = np.ones([env.observation_space.n,env.action_space.n])
# Set learning parameters
gamma = 0.95
epsilon = 1.0
epsilon_decay = 0.001
num_episodes = 1000
#create lists to contain total rewards and steps per episode
#jList = []
rList = []
for i in range(num_episodes):
    #Reset environment and get first new observation
    s, _ = env.reset()
    rAll = 0
    done = False # fall in H
    #The Q-Table learning algorithm
    while not done:
        rand = np.random.random()
        if rand < epsilon or not np.max(Q[s]) > 0:
            # else select ramdomly
            a = env.action_space.sample()
        else:
            #Choose an action by greedily (with noise) picking from Q table
            a = np.argmax(Q[s])
            
        #Get new state and reward from environment
        s1, r, done, _, _ = env.step(a)

        #Update Q-Table with new knowledge
        alpha = 1.0 / visits[s, a]
        Q[s,a] = (1-alpha)*Q[s,a] + alpha*(r+gamma*np.max(Q[s1,:]))

        rAll += r
        s = s1
        visits[s, a] += 1
        epsilon = max(epsilon - epsilon_decay, 0.0)

    #jList.append(j)
    rList.append(rAll)

In [80]:
print("Final Q-Table Values")
print(Q)

Final Q-Table Values
[[0.00000000e+00 0.00000000e+00 5.65623724e-01 6.34954599e-05]
 [2.16744974e-04 0.00000000e+00 5.95512426e-01 1.58412821e-03]
 [3.79013274e-03 7.49672585e-01 5.54764911e-04 9.98919211e-03]
 [2.10226703e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 7.89156560e-01 0.00000000e+00 8.91085995e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 9.02500000e-01 1.62901250e-01 0.00000000e+00]
 [8.57375000e-01 7.33135213e-01 0.00000000e+00 1.53682958e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 9.50000000e-01 0.00000000e+00]
 [9.02500000e-01 3.26562500e-01 1.00000000e+00 6.90755631e-01]
 [0.00000000e+00 0.00000000e+00 0.

In [81]:
test(env, 100, Q)

Success rate = 100.0%


# docs:
 * https://www.cs.swarthmore.edu/~meeden/cs63/f11/ml-ch13.pdf
 * http://neuro.cs.ut.ee/demystifying-deep-reinforcement-learning/